<a href="https://colab.research.google.com/github/Sidra-memon/Email_Generator_Using_Langchain/blob/main/Email_Generator_Using_Groq_and_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-groq langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 

In [3]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import uuid
import chromadb
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

st.title("Cold Email Generator")

url = st.text_input("Enter Job Posting URL")
submit_button = st.button("Generate Email")

if submit_button and url:
    llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile", groq_api_key="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")
    loader = WebBaseLoader(url)
    data = loader.load().pop().page_content
    #st.success("Job posting loaded!")

    prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
    )
    chain_extract = prompt_extract | llm
    res = chain_extract.invoke({"data": data})
    json_parser = JsonOutputParser()
    json_res = json_parser.parse(res.content)
    #st.success("Job details extracted!")
    df=pd.read_csv("/content/my_portfolio.csv")
    client = chromadb.PersistentClient('vectorstore')
    collection = client.get_or_create_collection(name="portfolio")
    if not collection.count():
      for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
    job=json_res
    links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])


    prompt_email = PromptTemplate.from_template(
             """
         ### JOB DESCRIPTION:
    {job_description}
  Your job is to write a cold email to the client regarding the job mentioned above describing your capability
  to fulfill their needs.
  Also add the most relevant ones from the following links to showcase portfolio: {link_list}
  Remember you are Sidra, ML Engineer at XYZ company.
  Avoid generic introductions—focus on **value, relevance, and engagement**.
  ### EMAIL (NO PREAMBLE):
    """
    )
    chain_email = prompt_email | llm
    res = chain_email.invoke({"job_description": str(job), "link_list": links})
    st.text_area("Generated Email", res.content, height=300)

    # Sidebar with instructions
st.sidebar.markdown("## Guide")
st.sidebar.info(
    "It allows users to input the URL of a company's careers page. "
    "The tool then extracts job listings from that page and generates personalized cold emails. "
    "These emails include relevant portfolio links sourced from a vector database, based on the specific job descriptions."
)



Writing app.py


In [5]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [6]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

34.141.196.96


In [8]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼⠴your url is: https://empty-cameras-tickle.loca.lt
^C
